# Name : Souri Yaswanth Krishna
# Reg.No : 19bci7070
# Course : CSE4006 LAB-9 Qs-3
# Motion-cosegmentation Using Deepfake COncept this based on firts order model deep_fakes

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjmMszv657v99U-l6Rru3VmJDyjXc49u0IMRpIMPkKeCacwbgb_8XE
Mounted at /content/drive


In [2]:
!pip install PyYAML==5.3.1
!git clone https://github.com/AliaksandrSiarohin/first-order-model

     |████████████████████████████████| 269 kB 4.3 MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44635 sha256=3ab670f29ff6aea6d399817a9fefff06e54a945559b91b7173df6f36d0be6b12
  Stored in directory: /root/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built PyYAML
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'first-order-model'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 299 (delta 2), reused 2 (delta 0), pack-reused 293
Receiving objects: 100% (299/299), 72.15 MiB | 24.77 MiB/s, done.
Resolving deltas: 100% (153/153), done.


## First Order Model for Image Automation <a name="First-Order-Model-for-Image-Automation"></a> 
The problme thus far, when it came to building deepfakes, was the need for additional information. For example, if the movement of the head was to be mapped there was a need for specific subject facial landmarks and full-body mappings required subject pose-estimation.  In 2019 a research team from the University of Toronto introduced a paper called *First Order Model for Image Automation* at the NeuraIPS conference. This paper presented an impressive way to animate a source image given a driving video, without any additional information or annotation about the object to automate. This method outperfomed state of the art on all the benchmarks and works well on a variety of images. Apart from this, once the model was trained, it can be used for transfer learning and it can be applied to an arbitraty object of the same category. The source code for the project can be found on [Aliaksandr Siarohin Github Repo](https://aliaksandrsiarohin.github.io/first-order-model-website/)

This section explores the model the research team presented, explains how it works and clones it for use in this paper.

### Motion Extraction <a name="Motion-Extraction"></a> 
The purpose of the motion estimation module is to predict a dense motion field. The model assumes there exists an abstract reference frame and it independently estimates two transformations: from reference to source and from reference to driving. This choice allows the model to independently process source and driving frames. This is desired since, at test time the model receives pairs of the source image and driving frames sampled from a different video, which can be very different visually.

The motion extractor utilizes an autoencoder to detect keypoints and extracts first-order motion representation that consists of sparse keypoints and local affine transformations. 


<img src = 'https://drive.google.com/uc?id=1e_aM4XLvLkXBlx7ItJckix-tZRFGbJeO' >

In [3]:
## Import modules
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [4]:
## Load source and drive
source_image = imageio.imread('/content/tom_cruise.png')
driving_video = imageio.mimread('/content/christopher_nolan.mp4')

## Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
HTML(display(source_image, driving_video).to_html5_video())

In [5]:
cd /content/first-order-model

/content/first-order-model


In [6]:
## Create model
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='/content/first-order-model/config/vox-256.yaml', 
                            checkpoint_path='/content/drive/MyDrive/deep learning /vox-adv-cpk.pth.tar')

In [7]:
## Import modules
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

## Save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])

## Video can be downloaded from /content folder
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 444/444 [01:20<00:00,  5.50it/s]


In [8]:
## Absolute coordinates
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 444/444 [01:21<00:00,  5.47it/s]


In [10]:
## Lebron image
source_image = imageio.imread('/content/resized.jpg')
driving_video = imageio.mimread('/content/christopher_nolan.mp4', memtest=False)


## Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 444/444 [01:24<00:00,  5.25it/s]


In [11]:
## Video cropping with Ffmpeg
!ffmpeg -i /content/my_video.mp4 -s 256x256 -c:a copy result.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [12]:
## Lebron image
source_image = imageio.imread('/content/tom_cruise.png')
driving_video = imageio.mimread('result.mp4', memtest=False)


## Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 282/282 [00:51<00:00,  5.47it/s]
